In [1]:
import requests
import re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Fru_Veg_M1.zip' 

Streaming output truncated to the last 5000 lines.
  inflating: Fru_Veg_M1/train/Quince/im_406.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_407.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_408.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_410.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_411.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_412.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_413.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_414.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_415.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_416.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_417.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_418.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_419.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_420.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_421.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_422.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_423.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_424.jpg  
  inflating: Fru_Veg_M1/train/Quince/im_425.jpg  

In [3]:
xform = transforms.Compose([transforms.Resize((100,100)), transforms.ToTensor()])
dataset_test = datasets.ImageFolder('/content/Fru_Veg_M1/test', transform=xform)
dataset_train = datasets.ImageFolder('/content/Fru_Veg_M1/train', transform=xform)

In [4]:
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=4, shuffle=True)

model = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [5]:
device = torch.device('cuda:0')
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [6]:
def run_test(model):
    nsamples_test = len(dataset_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

def run_train(model, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [7]:
run_all(model, optimizer, scheduler, 5)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


epoch 0: train loss 1.4163 acc 0.6531, test loss 0.1314 acc 0.8681
epoch 1: train loss 0.5813 acc 0.8420, test loss 0.0992 acc 0.9022
epoch 2: train loss 0.4219 acc 0.8841, test loss 0.0791 acc 0.9199
epoch 3: train loss 0.3558 acc 0.8996, test loss 0.0734 acc 0.9268
epoch 4: train loss 0.2919 acc 0.9162, test loss 0.0772 acc 0.9297
